# Financial Time Series Generator

In [1]:
from datetime import date
from dataclasses import dataclass
from typing import Callable, Optional, Dict, List
import operator
from sqlalchemy import *
import sqlite3
import pandas as pd
engine = create_engine('sqlite+pysqlite:///data/interview.db', module=sqlite3)

## Time Series Equation Language

An equation describes the calculation at time $t$ for all stocks $i$ in the universe at time $t$. 

For example:

* $Marketcap_{i,t} = market.close_{i,t} * market.shares_{i,t}$
* $Earnings_{i, t} = \sum_{\tau=t-3q}^{t} fundamental.net\_income_{i,\tau}$ 
* $EarningsToPrice_{i,t} =  Earnings_{i,t} / Marketcap_{i,t}$
* $FiftyTwoWeekRange_{i,t} = (p_{i,t} - min(\{p_{i,t-51w},\ldots,p_{i,t}\})) / (max(\{p_{i,t-51w},\ldots,p_{i,t}\}) - min(\{p_{i,t-51w},\ldots,p_{i,t}\}))$

## Task

Write a program that evaluates an <i>abitrary</i> equation for a universe of stocks, drawing data from a market and fundamental database.

## Equation API

In [2]:
class Equation(object):
    """Supertype of all equation objects."""
    pass


@dataclass
class Market(Equation):
    """Describes how to query a market item for stocks at time `t`."""
    item: str  # one of 'close' or 'shares'
    periods: int = 1  # matches all periods in the range [t-periods+1, t] (inclusive) according to the frequency
    freq: str = "months"  # determines the frequency of selection when periods > 1 (one of 'months' or 'weeks')
    agg: Optional[str] = None  # the aggregation (by stock) to apply if periods > 1 (None, 'min' or 'max')


@dataclass
class Fundamental(Equation):
    """Describes how to query a fundamental item for stocks at time `t`."""
    item: str
    quarters: int = 1  # matches all quarters in the range [t-quarters+1, t] (inclusive)
    agg: Optional[str] = None  # the aggregation (by stock) to apply if quarters > 1 (None or 'sum')


@dataclass
class Ref(Equation):
    """References another equation."""
    name: str  # Equation name


@dataclass
class Op(Equation):
    """Applies a binary function to the results of the operands."""
    fn: Callable
    lhs: Equation
    rhs: Equation

## Equation objects

In [3]:
equations = {
    'marketcap': Op(operator.mul, Market('close'), Market('shares')),
    'earnings': Fundamental('net_income', quarters=4, agg='sum'),
    'e2p': Op(operator.truediv, Ref('earnings'), Ref('marketcap')),
    'range_52w': Op(operator.truediv,
                    Op(operator.sub, 
                       Market('close'),
                       Market('close', freq='weeks', periods=52, agg='min')),
                    Op(operator.sub,
                       Market('close', freq='weeks', periods=52, agg='max'),
                       Market('close', freq='weeks', periods=52, agg='min'))
                    )
}

## The `universe` table -- Effective date for stocks in investment universes (11000 rows)

In [4]:
pd.read_sql(
    "select * from universe where universe_name = 'BacktestUniverse' and primary_id in ('AAPL', 'AMZN', 'GOOG', 'IBM', 'AMAT') order by primary_id", 
    engine, parse_dates=['start_date', 'end_date'])

,universe_name,primary_id,company_name,start_date,end_date
0,BacktestUniverse,AAPL,APPLE COMPUTER INC,1995-12-31,2049-08-31
1,BacktestUniverse,AMAT,APPLIED MATLS INC,1995-12-31,2049-08-31
2,BacktestUniverse,AMZN,AMAZON COM INC,1997-09-30,2049-08-31
3,BacktestUniverse,GOOG,GOOGLE INC,2004-09-30,2049-08-31
4,BacktestUniverse,IBM,INTERNATIONAL BUSINESS MACHS,1995-12-31,2049-08-31


* `universe_name` (PK) -- The name of the investment universe (there are two in the database)
* `primary_id` (PK) -- The unique identifier of the stock over time
* `company_name` -- The name of the company issuing the stock (not relevant to this problem)
* `start_date` (PK) and `end_date` (PK) -- The dates on which the stock entered and left the universe (inclusive)

## The `market` data table -- Monthly and weekly market data observations for all stocks from 2017-present

In [5]:
pd.read_sql("""select date, primary_id, close, shares from market where primary_id = 'AAPL' and date >= '2019-01-04 00:00:00.000000' and date <= '2019-01-31 00:00:00.000000'""", 
            engine, parse_dates=['date'])

,date,primary_id,close,shares
0,2019-01-04,AAPL,148.259995,4745.0
1,2019-01-11,AAPL,152.289993,4730.0
2,2019-01-18,AAPL,156.820007,4730.0
3,2019-01-25,AAPL,157.759995,4730.0
4,2019-01-31,AAPL,166.440002,4715.0


* `date` (PK) -- The date of the market observation (Fridays and last days of the months)
* `primary_id` (PK) -- The unique identifier of the stock over time
* `close` -- The end-of-day price for the stock (in dollars)
* `shares` -- The number of shares outstanding (in millions of shares)

## The `fundamental` table -- Quarterly financial statement data for all stocks from 2017-present

In [6]:
pd.read_sql("""select * from fundamental where primary_id = 'AAPL' and date = '2019-03-31 00:00:00.000000'""", engine, parse_dates=['date'])

,date,primary_id,fpe_offset,total_revenue,total_operating_expense,net_income,total_assets,total_liabilities,total_equity
0,2019-03-31,AAPL,-3,61137.0,45243.0,13822.0,367502.0,240624.0,126878.0
1,2019-03-31,AAPL,-2,53265.0,40653.0,11519.0,349197.0,234248.0,114949.0
2,2019-03-31,AAPL,-1,62900.0,46782.0,14125.0,365725.0,258578.0,107147.0
3,2019-03-31,AAPL,0,84310.0,60964.0,19965.0,373719.0,255827.0,117892.0


* `date` (PK) -- The date of the quarterly observation (not necessarily the announcement date)
* `primary_id` (PK) -- The unique identifier of the stock over time
* `fpe_offset` (PK) -- Offset from the most recent period between 0 and -4 (0=most recent, -1=previous quarter, -2=two quarters ago, etc...) 
* the remaining columns correspond to top-level financial statement items

## Helper Functions

In [7]:
def to_sqlite_str(d: date) -> str:
    """Converts date to a sqlite str parameter (sqlite does not have a native DATE type)."""
    return d.strftime('%Y-%m-%d 00:00:00.000000')


def weeks(end: date, num: int) -> List[date]:
    """Returns list of `num` Fridays up to the supplied date (inclusive, if a Friday)"""
    freq = pd.offsets.CustomBusinessDay(n=1, weekmask='Fri')
    return list(d.date() for d in pd.date_range(end=end, periods=num, freq=freq))


def months(end: date, num: int) -> List[date]:
    """Returns list of `num` month end dates up to the supplied date (inclusive, if a month end)"""
    freq = pd.offsets.MonthEnd(n=1)
    return list(d.date() for d in pd.date_range(end=end, periods=num, freq=freq))

### DB Query Examples

In [8]:
# Query using Pandas
_ = pd.read_sql("select * from market limit 10", engine, parse_dates=['date'])

# Query using sqlalchemy (returns an iterable of tuples)
_ = engine.execute("select * from market where date = :date limit 10", date=date(2019, 1, 31))

# Querying with a parameter to IN:
query = text("select * from market where date in :dates").bindparams(bindparam("dates", expanding=True))
_ = engine.execute(query, dates=[to_sqlite_str(d) for d in [date(2019, 1, 31), date(2019, 2, 28)]])

## Solution

In [9]:
from typing import Dict, List
from datetime import date

def calculate_t(name: str, universe: str, t: date, equations: Dict[str, Equation]):
    """
    Calculates the equation `name` that appears in the supplied `equations` dictionary for the `universe` at time `t`. 
    
    :name      maps to an equation in `equation`
    :universe  the name of a universe in the universe table
    :t         an end-of-quarter date (for example, date(2019, 3, 31))
    :equations complete dictionary of equations, where any Ref refers to a name in the dictionary
    :returns   a vector of values, one value for each name in the `universe` at time `t`
    """
    pass

In [10]:
calculate_t('marketcap', 'BacktestUniverse', date(2019, 3, 31), equations)

In [11]:
calculate_t('e2p', 'BacktestUniverse', date(2019, 3, 31), equations)

In [12]:
calculate_t('range_52w', 'BacktestUniverse', date(2019, 3, 31), equations)